In [7]:
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from pycaret.clustering import *
import nltk


In [8]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import re
import json
from urllib.request import Request, urlopen
from lxml import html
import time
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from PIL import Image
from io import BytesIO
import cv2
from unicodedata import normalize
import string
from nltk.stem import PorterStemmer

In [9]:
path_to_export = '/Users/henricobela/Desktop/Estudos/Github/Faculdade/FIAP/Segundo_ano/2o_semestre/AUTOML/Checkpoint_2_webapp_recomendacao/data' #mac
df = pd.read_csv(f'{path_to_export}/all_movies.csv', sep = ';')

df = df[["title_pt", "title_en", "year", "rating", "genre", "sinopse"]]

print(df.shape)
df.head()

(357, 6)


,title_pt,title_en,year,rating,genre,sinopse
0,Besouro Azul,1. Besouro Azul,2023,6.8 (17K),Ação,"Jaime Reyes, um adolescente de origem mexicana..."
1,Barbie,2. Barbie,2023,7.4 (255K),Aventura,Viver na Terra da Barbie é ser um ser perfeito...
2,Oppenheimer,3. Oppenheimer,2023,8.6 (381K),Biografia,A história do cientista americano J. Robert Op...
3,As Tartarugas Ninja: Caos Mutante,4. As Tartarugas Ninja: Caos Mutante,2023,7.5 (19K),Animação,Os irmãos Tartaruga trabalham para conquistar ...
4,Ruim Pra Cachorro,5. Ruim Pra Cachorro,2023,6.4 (3.2K),Animação,Um cachorro abandonado se une a outros vira-la...


In [10]:
df['year'] = df['year'].astype('int64')
df["rating"] = df["rating"].apply(lambda x: x.split("\xa0")[0] if not isinstance(x, float) else x)
idx = df[df.rating.isin(["Rate"])].index.tolist()
df.loc[idx, "rating"] = 6.0
df['rating'] = df['rating'].astype('float64')
df.rating.fillna(6.0, inplace = True)
df.dropna(inplace = True)
df

,title_pt,title_en,year,rating,genre,sinopse
0,Besouro Azul,1. Besouro Azul,2023,6.8,Ação,"Jaime Reyes, um adolescente de origem mexicana..."
1,Barbie,2. Barbie,2023,7.4,Aventura,Viver na Terra da Barbie é ser um ser perfeito...
2,Oppenheimer,3. Oppenheimer,2023,8.6,Biografia,A história do cientista americano J. Robert Op...
3,As Tartarugas Ninja: Caos Mutante,4. As Tartarugas Ninja: Caos Mutante,2023,7.5,Animação,Os irmãos Tartaruga trabalham para conquistar ...
4,Ruim Pra Cachorro,5. Ruim Pra Cachorro,2023,6.4,Animação,Um cachorro abandonado se une a outros vira-la...
...,...,...,...,...,...,...
352,Meus Sogros Tão pro Crime,Meus Sogros Tão pro Crime,2023,5.4,Ação,Um conservador gerente de banco prestes a se c...
353,Dunkirk,Dunkirk,2017,7.8,Ação,"Soldados aliados da Bélgica, do Império Britân..."
354,Guardiões da Galáxia,Guardiões da Galáxia,2014,8.0,Ação,Um grupo de criminosos intergalácticos deve se...
355,Titanic,Titanic,1997,7.9,Drama,Uma aristocrata de dezessete anos se apaixona ...


In [11]:
df_processed = df.copy()
df_processed['sinopse'] = df_processed['sinopse'].str.lower()
df_processed['sinopse'].head()

0    jaime reyes, um adolescente de origem mexicana...
1    viver na terra da barbie é ser um ser perfeito...
2    a história do cientista americano j. robert op...
3    os irmãos tartaruga trabalham para conquistar ...
4    um cachorro abandonado se une a outros vira-la...
Name: sinopse, dtype: object

In [12]:
def qty_words(text):
    words = text.split()
    word_count = len(words)
    return word_count

In [13]:
df_processed['word_count'] = df_processed['sinopse'].apply(qty_words).astype('int64')
df_processed.head()

,title_pt,title_en,year,rating,genre,sinopse,word_count
0,Besouro Azul,1. Besouro Azul,2023,6.8,Ação,"jaime reyes, um adolescente de origem mexicana...",24
1,Barbie,2. Barbie,2023,7.4,Aventura,viver na terra da barbie é ser um ser perfeito...,29
2,Oppenheimer,3. Oppenheimer,2023,8.6,Biografia,a história do cientista americano j. robert op...,17
3,As Tartarugas Ninja: Caos Mutante,4. As Tartarugas Ninja: Caos Mutante,2023,7.5,Animação,os irmãos tartaruga trabalham para conquistar ...,19
4,Ruim Pra Cachorro,5. Ruim Pra Cachorro,2023,6.4,Animação,um cachorro abandonado se une a outros vira-la...,15


In [14]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henricobela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df_processed['sinopse_no_stopwords'] = df_processed['sinopse'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [16]:


from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()



In [17]:


def stem(text):
    y= []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)



In [18]:
df_processed["sinopse_no_stopwords"] = df_processed["sinopse_no_stopwords"].apply(stem)

In [19]:
df_processed

,title_pt,title_en,year,rating,genre,sinopse,word_count,sinopse_no_stopwords
0,Besouro Azul,1. Besouro Azul,2023,6.8,Ação,"jaime reyes, um adolescente de origem mexicana...",24,"jaim reyes, adolescent origem mexicana encontr..."
1,Barbie,2. Barbie,2023,7.4,Aventura,viver na terra da barbie é ser um ser perfeito...,29,viver terra barbi perfeito lugar perfeito. men...
2,Oppenheimer,3. Oppenheimer,2023,8.6,Biografia,a história do cientista americano j. robert op...,17,história cientista americano j. robert oppenhe...
3,As Tartarugas Ninja: Caos Mutante,4. As Tartarugas Ninja: Caos Mutante,2023,7.5,Animação,os irmãos tartaruga trabalham para conquistar ...,19,irmão tartaruga trabalham conquistar amor cida...
4,Ruim Pra Cachorro,5. Ruim Pra Cachorro,2023,6.4,Animação,um cachorro abandonado se une a outros vira-la...,15,cachorro abandonado une outro vira-lata vingar...
...,...,...,...,...,...,...,...,...
352,Meus Sogros Tão pro Crime,Meus Sogros Tão pro Crime,2023,5.4,Ação,um conservador gerente de banco prestes a se c...,33,conservador gerent banco prest casar amor vida...
353,Dunkirk,Dunkirk,2017,7.8,Ação,"soldados aliados da bélgica, do império britân...",25,"soldado aliado bélgica, império britânico fran..."
354,Guardiões da Galáxia,Guardiões da Galáxia,2014,8.0,Ação,um grupo de criminosos intergalácticos deve se...,20,grupo criminoso intergaláctico deve unir imped...
355,Titanic,Titanic,1997,7.9,Drama,uma aristocrata de dezessete anos se apaixona ...,20,aristocrata dezesset ano apaixona artista gent...


In [20]:


from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=355)



In [21]:
vectors=cv.fit_transform(df_processed["sinopse_no_stopwords"]).toarray()

In [22]:
cv.get_feature_names_out()

array(['acabar', 'acidente', 'acusado', 'adolescent', 'advogado', 'agent',
       'agora', 'ajuda', 'ajudar', 'ajudá', 'aldeia', 'alemã', 'alemão',
       'algo', 'aliado', 'ameaça', 'americano', 'amigo', 'amigos',
       'amizad', 'amor', 'ano', 'anos', 'ant', 'antigo', 'aposentado',
       'apó', 'aranha', 'aristocrata', 'arma', 'assassinar',
       'assassinato', 'assassino', 'atravé', 'aventura', 'bandido',
       'boxeador', 'brinquedo', 'britânico', 'bruxa', 'busca', 'cachorro',
       'cada', 'caminho', 'campo', 'cao', 'carro', 'casa', 'casal',
       'caso', 'cavaleiro', 'caçador', 'centro', 'chamada', 'chamado',
       'chefe', 'chega', 'cidad', 'cidade', 'cientista', 'cinco',
       'cinema', 'clandestino', 'coisa', 'colega', 'coloca', 'começa',
       'começar', 'comum', 'confrontar', 'confronto', 'confundido',
       'conhec', 'conhecido', 'consegu', 'conta', 'contra', 'contratado',
       'control', 'coringa', 'corpo', 'corrupção', 'criança', 'crime',
       'criminoso', '

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [24]:
def recommend(movie):
    movie_index = df_processed[df_processed["title_pt"] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True, key= lambda x:x[1])[1:6] 
    for i in movie_list:
        print(df_processed.iloc[i[0]].title_pt)

In [25]:
recommend("Dunkirk")

Intocáveis
Rush: No Limite da Emoção
Cafarnaum
O Rei Leão
Antes do Pôr do Sol


In [26]:
df_processed

,title_pt,title_en,year,rating,genre,sinopse,word_count,sinopse_no_stopwords
0,Besouro Azul,1. Besouro Azul,2023,6.8,Ação,"jaime reyes, um adolescente de origem mexicana...",24,"jaim reyes, adolescent origem mexicana encontr..."
1,Barbie,2. Barbie,2023,7.4,Aventura,viver na terra da barbie é ser um ser perfeito...,29,viver terra barbi perfeito lugar perfeito. men...
2,Oppenheimer,3. Oppenheimer,2023,8.6,Biografia,a história do cientista americano j. robert op...,17,história cientista americano j. robert oppenhe...
3,As Tartarugas Ninja: Caos Mutante,4. As Tartarugas Ninja: Caos Mutante,2023,7.5,Animação,os irmãos tartaruga trabalham para conquistar ...,19,irmão tartaruga trabalham conquistar amor cida...
4,Ruim Pra Cachorro,5. Ruim Pra Cachorro,2023,6.4,Animação,um cachorro abandonado se une a outros vira-la...,15,cachorro abandonado une outro vira-lata vingar...
...,...,...,...,...,...,...,...,...
352,Meus Sogros Tão pro Crime,Meus Sogros Tão pro Crime,2023,5.4,Ação,um conservador gerente de banco prestes a se c...,33,conservador gerent banco prest casar amor vida...
353,Dunkirk,Dunkirk,2017,7.8,Ação,"soldados aliados da bélgica, do império britân...",25,"soldado aliado bélgica, império britânico fran..."
354,Guardiões da Galáxia,Guardiões da Galáxia,2014,8.0,Ação,um grupo de criminosos intergalácticos deve se...,20,grupo criminoso intergaláctico deve unir imped...
355,Titanic,Titanic,1997,7.9,Drama,uma aristocrata de dezessete anos se apaixona ...,20,aristocrata dezesset ano apaixona artista gent...


In [27]:
import pickle as pk
pk.dump(df_processed.to_dict() ,open("./model/df.pkl","wb"))
pk.dump(similarity ,open("./model/model.pkl","wb"))

In [28]:


url1 = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'


In [29]:


url2 = 'https://www.imdb.com/chart/moviemeter/?ref_=chtbo_ql_2'


In [30]:


url3 = 'https://www.imdb.com/chart/boxoffice/'


In [31]:
userAgents=[
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"]

In [32]:
headers = {
          'authority': 'www.amazon.com.br',
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
          'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
          'cache-control': 'max-age=0',
          'device-memory': '8',
          'downlink': '10',
          'dpr': '1.875',
          'ect': '4g',
          'rtt': '50',
          'sec-ch-device-memory': '8',
          'sec-ch-dpr': '1.875',
          'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
          'sec-ch-ua-mobile': '?0',
          'sec-ch-ua-platform': '"Windows"',
          'sec-ch-ua-platform-version': '"10.0.0"',
          'sec-ch-viewport-width': '455',
          'sec-fetch-dest': 'document',
          'sec-fetch-mode': 'navigate',
          'sec-fetch-site': 'none',
          'sec-fetch-user': '?1',
          'upgrade-insecure-requests': '1',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
          'viewport-width': '455',
          }

In [33]:
list_title_pt = []
list_images = []
list_links = []

for url in [url1, url2, url3]:

    response = requests.get(url, headers={"User-agent": userAgents[1]})
    html = response.text
    bs = BeautifulSoup(html)
    image_links = bs.find_all("div", attrs ={"class":"ipc-media__img"})
    

    for i in image_links:
        images = i.find("img").get("src")
        list_images.append(images)
    
    for a in bs.find_all('a', href=True):
        if '/title/' in a['href'] and 'https://www.imdb.com/'+ a['href'] not in list_links:
            list_links.append(('https://www.imdb.com/'+a['href'])[:-15])


            
#Remove duplicates
list_links = list(dict.fromkeys(list_links))
#Deleting first element
list_links = list_links[1:]

for link in list_links:
    time.sleep(.5)
    response = requests.get(link, headers = headers)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    try:
        for x in soup.find('title'):
            #title_pt
            title_pt = (x.text)[:-14].strip()
            list_title_pt.append(title_pt)
    except:
        list_title_pt.append(np.nan)


In [34]:
import base64

image_arrays = []
for img_url in list_images:
    if img_url:
        img_response = requests.get(img_url)
        if img_response.status_code == 200:
            img_bytes = BytesIO(img_response.content)
            image_arrays.append(img_bytes.getvalue())

In [59]:
list_images

['https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_QL75_UX140_CR0,1,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BZjgwNzE5ODgtYzAyZC00YTZhLThhNDktMDVlOGNhYzk5NTVkXkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UY207_CR3,0,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BM2E1ZjEyYWQtMjgyMy00ZTkwLThkM2QtN2YwM2NmYzAyYjE0XkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UY207_CR8,0,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BOGM2M2Q0ZWQtZjc5NC00NDY4LWIxYTktMDE4ZjIyYTRlNTIyXkEyXkFqcGdeQXVyODc0OTEyNDU@._V1_QL75_UY207_CR4,0,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BODRmNWRiYmUtNGM2Yy00NTgwLTgxNzMtMGExYzI5NjllNjBhXkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UY207_CR3,0,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BYjM5ZjYxNjQtZjdhYi00NzBiLTgyNGItOGU3YTYwYWE0NmEzXkEyXkFqcGdeQXVyMTAzMDM4MjM0._V1_QL75_UX140_CR0,0,140,207_.jpg',
 'https://m.media-amazon.com/images/M/MV5BNzZiOTI4MWItZGMxZC00NjZkLWJl

In [35]:
len(list_title_pt)

355

In [36]:
len(image_arrays)

360

In [37]:
df = pd.DataFrame({'title_pt': list_title_pt[:355],}
                #    'poster_array': image_arrays[:355]}
                #    ,
                   )
df

,title_pt,poster_array
0,Um Sonho de Liberdade,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
1,O Poderoso Chefão,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
2,Batman: O Cavaleiro das Trevas,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3,O Poderoso Chefão II,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4,12 Homens e uma Sentença,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...
350,As Tartarugas Ninja: Caos Mutante,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
351,Bottoms,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
352,Megatubarão 2,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
353,Ruim Pra Cachorro,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [38]:
df.to_excel("data/imgs.xlsx")

In [104]:
import requests
from bs4 import BeautifulSoup

# Título do filme que você deseja procurar
titulo_filme = "1. Um Sonho de Liberdade"

# Construa a URL da pesquisa no IMDb
url_pesquisa = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'

# Faça a solicitação HTTP
# response = requests.get(url_pesquisa, headers = headers)

response = requests.get(link, headers = headers)


# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Analise a página com BeautifulSoup
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    
    # Encontre o elemento que contém o título do filme (por exemplo, um link <a>)
    link_filme = soup.find('title')
    
    if link_filme:
        # Encontre o elemento da imagem do filme dentro do mesmo bloco HTML, se possível
        # Isso depende da estrutura específica da página
        # Suponha que a imagem esteja em uma div com a classe 'poster', por exemplo
        div_poster = link_filme.find_next('div')
        
        if div_poster:
            # Obtenha a URL da imagem
            imagem_url = div_poster.find('img')['src']
            
            # Faça o que você deseja com a URL da imagem
            if img_url:
                img_response = requests.get(img_url)

                if img_response.status_code == 200:
                    imagem = Image.open(BytesIO(img_response.content))
                
                # Mostre a imagem ou faça o que desejar com ela
                    imagem.show()
        else:
            print("Imagem não encontrada para este filme.")
    else:
        print("Título do filme não encontrado na página.")
else:
    print("Falha na solicitação HTTP.")


In [96]:
link_filme.find_next('div')

<div> <img alt="" height="1" onload="window.ue_sbl &amp;&amp; window.ue_sbl();" src="//fls-na.amazon.com/1/batch/1/OP/A1EVAM02EL8SFB:131-0939883-7382106:H5JX1C6FEKH9EABTRP60$uedata=s:%2Fuedata%2Fuedata%3Fstaticb%26id%3DH5JX1C6FEKH9EABTRP60:0" style="display:none;visibility:hidden;" width="1"/>
<script>
!function(){function n(n,t){var r=i(n);return t&&(r=r("instance",t)),r}var r=[],c=0,i=function(t){return function(){var n=c++;return r.push([t,[].slice.call(arguments,0),n,{time:Date.now()}]),i(n)}};n._s=r,this.csa=n}();;
csa('Config', {});
if (window.csa) {
    csa("Config", {
        
        'Events.Namespace': 'csa',
        'ObfuscatedMarketplaceId': 'A1EVAM02EL8SFB',
        'Events.SushiEndpoint': 'https://unagi.amazon.com/1/events/com.amazon.csm.csa.prod',
        'CacheDetection.RequestID': "H5JX1C6FEKH9EABTRP60",
        'CacheDetection.Callback': window.ue && ue.reset,
        'LCP.elementDedup': 1,
        'lob': '0'
    });

    csa("Events")("setEntity", {
        page: {re

In [106]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

# Título do filme que você deseja procurar
titulo_filme = "1. Um Sonho de Liberdade"

# Construa a URL da pesquisa no IMDb Top 250
url_pesquisa = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'

# Faça a solicitação HTTP
response = requests.get(url_pesquisa, headers = headers)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Analise a página com BeautifulSoup
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    
    # Encontre o elemento que contém o título do filme
    titulo_element = soup.find('a', text=titulo_filme)
    
    if titulo_element:
        # Encontre o elemento da imagem do filme dentro do mesmo bloco HTML, se possível
        # Suponha que a imagem esteja dentro de uma div com a classe 'poster', por exemplo
        div_poster = titulo_element.find_next('div')
        
        if div_poster:
            # Verifique se há uma tag 'img' dentro de div_poster
            img = div_poster.find('img')
            
            if img:
                # Obtenha a URL da imagem
                imagem_url = img.get('src')
                
                # Faça o download da imagem
                img_response = requests.get(imagem_url)
                
                # Verifique se o download foi bem-sucedido
                if img_response.status_code == 200:
                    # Abra a imagem usando o Pillow a partir dos bytes obtidos
                    imagem = Image.open(BytesIO(img_response.content))
                    
                    # Mostre a imagem ou faça o que desejar com ela
                    imagem.show()
                else:
                    print("Falha ao baixar a imagem.")
            else:
                print("Imagem não encontrada para este filme.")
        else:
            print("Div do pôster não encontrada para este filme.")
    else:
        print(f"Filme '{titulo_filme}' não encontrado na lista IMDb Top 250.")
else:
    print("Falha na solicitação HTTP.")


Imagem não encontrada para este filme.
